In [1]:
import sys
from pathlib import Path

In [2]:
import numpy as np
import pandas as pd

In [3]:
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F

Load data

In [4]:
feature_dir = Path.cwd().parent.joinpath("data").joinpath("interm")
training_set_dir = feature_dir.joinpath("single_track")
X_file = training_set_dir.joinpath("X.npy")
Y_file = training_set_dir.joinpath("Y.npy")

X = np.load(X_file)
Y = np.load(Y_file)
X = torch.tensor(X.reshape(100,1,256,256), dtype=torch.float)
Y = torch.tensor(Y.reshape(100,1,32,32), dtype=torch.float)

X_train = X[0:70]
Y_train = Y[0:70]
X_val = X[70:100]
Y_val = Y[70:100]

In [5]:
class rpn(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(4, 4, 3, padding=1)
        
        self.conv3 = nn.Conv2d(4, 8, 3, padding=1)
        self.conv4 = nn.Conv2d(8, 8, 3, padding=1)
        
        self.conv5 = nn.Conv2d(8, 16, 3, padding=1)
        self.conv6 = nn.Conv2d(16, 16, 3, padding=1)
        
        self.conv7 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv8 = nn.Conv2d(32, 32, 3, padding=1)
        
        self.conv9 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv10 = nn.Conv2d(32, 32, 3, padding=1)
        
        self.conv11 = nn.Conv2d(32, 16, 3, padding=1)
        self.conv12 = nn.Conv2d(16, 8, 3, padding=1)
        self.conv13 = nn.Conv2d(8, 4, 3, padding=1)
        self.conv14 = nn.Conv2d(4, 1, 3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = self.pool(x)
        
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv9(x))
        x = F.relu(self.conv10(x))
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.relu(self.conv13(x))
        x = torch.sigmoid(self.conv14(x))
        
        return x
net = rpn()

In [6]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [7]:
def weighted_binary_cross_entropy(output, target, weights=None):
        
    if weights is not None:
        assert len(weights) == 2
        
        loss = weights[1] * (target * torch.log(output)) + \
               weights[0] * ((1 - target) * torch.log(1 - output))
    else:
        loss = target * torch.log(output) + (1 - target) * torch.log(1 - output)

    return torch.neg(torch.mean(loss))

In [10]:
for epoch in range(100):
    
    loss_train = 0.0
    
    for i, (x, y) in enumerate(zip(X_train,Y_train)):
        optimizer.zero_grad()
        
        p = net(x.reshape(1,1,256,256)).reshape(1,32,32)
        loss = weighted_binary_cross_entropy(p, y.reshape(1,32,32), weights=[1, 500])
        #loss = criterion(p,y.reshape(1,32,32))
        loss.backward()
        optimizer.step()
        
        loss_train += loss.item()
        
    correct = 0
    map_precisions =[]
    for x, y in zip(X_train, Y_train):
        p = net(x.reshape(1,1,256,256))
        p = (p>0.5).float()
        if torch.all(p==y):
            correct += 1
        map_precisions.append(torch.sum(p==y)/1024)
    precision_train = correct/70.0
    map_precision_avg = np.average(map_precisions)

    loss_val = 0.0
    for x, y in zip(X_val, Y_val):
        p = net(x.reshape(1,1,256,256)).reshape(1,32,32)
        loss = weighted_binary_cross_entropy(p,y.reshape(1,32,32), weights=[1, 500])
        #loss = criterion(p,y.reshape(1,32,32))
        loss_val += loss.item()
        
    print(f"Epoch {epoch+1}: train loss: {loss_train/70:.3f}, val loss: {loss_val/30:.3f}")
    print(f"map precision: {map_precision_avg}")
            

Epoch 1: train loss: 1.235, val loss: 1.167
map precision: 0.0009765625
Epoch 2: train loss: 1.117, val loss: 1.074
map precision: 0.0009765625
Epoch 3: train loss: 1.044, val loss: 1.019
map precision: 0.9990234375
Epoch 4: train loss: 1.002, val loss: 0.988
map precision: 0.9990234375
Epoch 5: train loss: 0.978, val loss: 0.969
map precision: 0.9990234375
Epoch 6: train loss: 0.963, val loss: 0.958
map precision: 0.9990234375
Epoch 7: train loss: 0.954, val loss: 0.951
map precision: 0.9990234375
Epoch 8: train loss: 0.949, val loss: 0.947
map precision: 0.9990234375
Epoch 9: train loss: 0.946, val loss: 0.945
map precision: 0.9990234375
Epoch 10: train loss: 0.944, val loss: 0.944
map precision: 0.9990234375
Epoch 11: train loss: 0.943, val loss: 0.943
map precision: 0.9990234375
Epoch 12: train loss: 0.943, val loss: 0.942
map precision: 0.9990234375
Epoch 13: train loss: 0.942, val loss: 0.942
map precision: 0.9990234375
Epoch 14: train loss: 0.942, val loss: 0.942
map precision: 